In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [4]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       -2
pinpir       13
connection1  BV6000,bubblino,192.168.43.1

Sent 7 lines (173 bytes) to config.txt.


In [3]:
%sendtofile --quiet --source utils.py
%sendtofile --quiet --source stdmqttas.py

Sent 32 lines (969 bytes) to utils.py.
Sent 76 lines (2732 bytes) to stdmqttas.py.


In [8]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [10]:
%sendtofile main.py

import time, sys, machine, itertools, ustruct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from stdmqttas import pinled, flashpinled, shortmac, topicstem, topicstatus

import network
network.WLAN().active(0)  # disable the connection at startup
pinpir = machine.Pin(int(fconfig["pinpir"]), machine.Pin.IN)
flashpinled(5, 300, 300)

print("topicstem", topicstem)

topicpir = topicstem+"/pir"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"

async def pirchangedetectiontask(client):
    pirstate = -2
    for n in itertools.count():
        prevpirstate, pirstate = pirstate, pinpir.value()
        if pirstate != prevpirstate:
            if client.isconnected():
                await client.publish(topicpir, str(pirstate), retain=True)
                blinkstate = ((n%400) == 0)
                pinled.value((1-pirstate) != blinkstate)
            else:
                pirstate = -2
        await asyncio.sleep_ms(5)
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

    
async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
        
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(pirchangedetectiontask(client))
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()


Sent 48 lines (1631 bytes) to main.py.


In [6]:
import stdmqttas

In [7]:
print(dir(stdmqttas))

['__class__', '__name__', 'Pin', 'config', 'network', 'time', 'asyncio', 'fconfig', 'mqttconnecttask', 'callbackcmdtask', 'slowflashpinled']


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

In [ ]:
"{:02X}{:02X}{:02X}".format(*network.WLAN().config('mac')[-3:])